In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import mxnet as mx
from mxnet import gluon
import pickle
import json
import random
import inspect
from scipy import stats
from sklearn.metrics import mean_squared_error
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pathlib import Path
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepstate import DeepStateEstimator
from gluonts.trainer import Trainer
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator, MultivariateEvaluator
from gluonts.distribution.multivariate_gaussian import MultivariateGaussianOutput
from gluonts.model.predictor import Predictor
from gluonts.model.prophet import ProphetPredictor
from gluonts.model.r_forecast import RForecastPredictor
from indycar.model.NaivePredictor import NaivePredictor
from indycar.model.ZeroPredictor import ZeroPredictor
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

#from indycar.model.stint_predictor_fastrun import *
import indycar.model.stint_simulator_shortterm as stint

INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU


In [3]:
def simulation(datasetid, testevent, taskid, runts, expid, predictionlen, datamode, loopcnt, featuremode = stint.FEATURE_STATUS):
    #
    # configurataion
    #
    # model path:  <_dataset_id>/<_task_id>-<trainid>/
    #_dataset_id = 'indy2013-2018-nocarid'
    stint.init()
    stint._dataset_id = datasetid
    stint._test_event = testevent
    #_test_event = 'Indy500-2019'

    stint._feature_mode = featuremode
    stint._context_ratio = 0.

    stint._task_id = taskid  # rank,laptime, the trained model's task
    stint._run_ts = runts   #COL_LAPTIME,COL_RANK
    stint._exp_id=expid  #rank, laptime, laptim2rank, timediff2rank... 

    stint._train_len = 40
    predictor = stint.load_model(predictionlen, 'oracle',trainid='2018')

    ret2 = {}
    for i in range(loopcnt):
        ret2[i] = stint.run_simulation_shortterm(predictor, predictionlen, stint.freq, datamode=datamode)

    acc = []
    for i in ret2.keys():
        df = ret2[i]
        _x = stint.get_evalret_shortterm(df)
        acc.append(_x)

    b = np.array(acc)
    print(np.mean(b, axis=0))
    
    return b, ret2

### test low mode

In [29]:
acc, ret = simulation('indy2013-2018', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,10)
for df in ret:
    stint.get_evalret_shortterm(df)

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
pred: acc=0.7074468085106383, mae=1.5666666666666667, rmse=10.982771535580524,r2=0.8466624982619189, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7021276595744681, mae=1.5857677902621723, rmse=11.110486891385769,r2=0.8448793824491824, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7180851063829787, mae=1.5801498127340825, rmse=10.99063670411985,r2=0.8465526876106946, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.6968085106382979, mae=1.5704119850187266, rmse=11.265168539325842,r2=0.8427197729751056, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7074468085106383, mae=1.546441947565543, rmse=10.67940074906367,r2=0.8508980519519969, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292
pred: acc=0.7127659574468085, mae=1.5760299625468164, rmse=11.097378277153558,r2=0.8450624002012

TypeError: 'int' object is not subscriptable

### test

In [36]:
np.mean(acc, axis=0)

array([ 0.70904255,  1.56734082, 10.99475655,  0.84649517])

In [12]:
139./188

0.7393617021276596

In [27]:
stint.get_evalret_shortterm(df)

pred: acc=0.6968085106382979, mae=1.5872659176029962,45.08510638297872, rmse=11.434831460674157,r2=0.8403510003558398, acc_naive=0.7180851063829787, mae_naive=1.3393258426966292, 38.04255319148936


(0.6968085106382979,
 1.5872659176029962,
 11.434831460674157,
 0.8403510003558398)

In [15]:
df

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
0,1,10,3.0,3.0,0.0,0,3.0,0.0,0
1,3,10,7.0,7.0,0.0,0,7.0,0.0,0
2,4,10,12.0,12.0,0.0,0,12.0,0.0,0
3,6,10,15.0,15.0,0.0,0,15.0,0.0,0
4,7,10,28.0,28.0,0.0,0,29.0,1.0,1
...,...,...,...,...,...,...,...,...,...
5335,60,197,15.0,15.0,0.0,0,15.0,0.0,0
5336,64,197,16.0,16.0,0.0,0,14.0,-2.0,-1
5337,66,197,10.0,10.0,0.0,0,16.0,6.0,1
5338,88,197,13.0,13.0,0.0,0,12.0,-1.0,-1


In [23]:
np.sum(df[['diff']].values.reshape(-1))

[autoreload of indycar.model.stint_simulator_shortterm failed: Traceback (most recent call last):
  File "/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 781, in get_code
  File "<frozen importlib._bootstrap_external>", line 741, in source_to_code
  File "<f

0.0